In [1]:
import os
import sys
import flowfairy
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import soundfile as sf
import IPython.display as display

In [2]:
sys.path.append('..')
os.environ.setdefault('GLUE_SETTINGS_MODULE', 'settings')

'settings'

## Load some data

In [3]:
folder = '/Users/jonas/aiprojects/audio_dataset/preprocessed_22050/test/'
all_data = os.listdir(folder)

def get(i):
    return np.load(os.path.join(folder, all_data[i]))

def audio(npz):    
    audio_file = str(npz['audio_file']).replace('//','/')
    sound, _ = sf.read(audio_file)
    return sound

## Setup the network

In [4]:
import netops

shape = [1, None, 1]

X = tf.placeholder(tf.float32, shape=shape)
X_noise = tf.placeholder(tf.float32, shape=shape)
y = tf.placeholder(tf.int64, shape=shape)
cls = tf.placeholder(tf.int32, shape=[None])
outputlen = tf.placeholder(tf.int32)


net = netops.Net()

with tf.variable_scope('network') as scope:
    with tf.name_scope('training'):
        tx, tnet, *_ = net.feedforward(X, X_noise, y, cls, is_training=True, outputlen=outputlen)
    scope.reuse_variables()

    with tf.name_scope('validation'):
        vx, vnet, *_ = net.feedforward(X, X_noise, y, cls, outputlen=outputlen)

x Tensor("network/training/concat_2:0", shape=(1, ?, 2), dtype=float32)
conv1 Tensor("network/training/glu_conv1_1/mul:0", shape=(1, ?, 1, 4), dtype=float32)
conv1_d1  Tensor("network/training/glu_conv1_d1/mul:0", shape=(1, ?, 1, 4), dtype=float32)
conv1_d2  Tensor("network/training/glu_conv1_d2/mul:0", shape=(1, ?, ?, 4), dtype=float32)
conv1_d4  Tensor("network/training/glu_conv1_d4/mul:0", shape=(1, ?, ?, 4), dtype=float32)
conv1_concat Tensor("network/training/concat_4:0", shape=(1, ?, 1, 12), dtype=float32)
d1 Tensor("network/training/dense_1/d_4/glu_glu/mul:0", shape=(1, ?, 1, 8), dtype=float32)
d2 Tensor("network/training/dense_2/d_4/glu_glu/mul:0", shape=(1, ?, 1, 16), dtype=float32)
d3 Tensor("network/training/dense_3/d_4/glu_glu/mul:0", shape=(1, ?, 1, 32), dtype=float32)
d4 Tensor("network/training/dense_4/d_4/glu_glu/mul:0", shape=(1, ?, 1, 16), dtype=float32)
d5 Tensor("network/training/dense_5/d_4/glu_glu/mul:0", shape=(1, ?, 1, 8), dtype=float32)
conv5:  Tensor("network/

## Start a session

In [5]:
session = tf.InteractiveSession()

## Restore the saved network

In [6]:
saver = tf.train.Saver()
network = ('denseU_sss_g3', 32000)
network = ('denseU_sss_g3', 32000)
network = 'denseU_ss_g2'
ckpt = 32000

saver.restore(session, f'../logs/{network_name}.ckpt-{ckpt}')#tf.train.latest_checkpoint('../logs', latest_filename=network_name))

## Run the network on the data

In [7]:
def norm(tensor):
    tmin = tf.reduce_min(tensor)
    return tf.div((tensor - tmin), (tf.reduce_max(tensor) - tmin) + 1e-12)


pred = tf.argmax(vnet, 2)
pred = norm(tf.cast(pred, tf.float32))

In [8]:
npz1 = get(0)
npz2 = get(1)
a1 = audio(npz1)
a2 = audio(npz2)

olen = min(a1.shape[0], a2.shape[0])

a1 = a1[None, :olen, None]
a2 = a2[None, :olen, None]

olen -= 22050 // 3400 + 10
# decrement outputlen until compatible with network
while olen % 4:
    olen -= 1


feed_dict = {X: a1, X_noise: a2, outputlen: olen}

noisy, prediction = session.run([tx, pred], feed_dict=feed_dict)

In [9]:
display.display(display.Audio(data=prediction[0], rate=22050))

In [10]:
display.display(display.Audio(data=noisy[:,:,0], rate=22050))

In [11]:
#print(npz1.files)